In [1]:
import transformers
import torch
import numpy as np
import gc
import tempfile

from transformers import (LlamaForCausalLM, 
                          LlamaTokenizer,
                          AutoTokenizer, 
                          AutoModelForCausalLM)

from typing import List, Dict, Any


/opt/conda/envs/amadeus/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 加载模型

In [2]:
model_path = '/workspace/acl/model_zoo/llama/llama-2-7b-hf'
tokenizer = AutoTokenizer.from_pretrained(model_path,trust_remote_code=False, device_map = "auto")
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


FileNotFoundError: No such file or directory: "/workspace/acl/model_zoo/llama/llama-2-7b-hf/model-00001-of-00002.safetensors"

### 加载模型参数

In [ ]:
def load_model(model_name_or_path, trust_remote_code:bool=True, device_map="auto"):
    model = AutoModelForCausalLM(model_name_or_path, trust_remote_code=trust_remote_code, device_map=device_map)
    return model

def get_model_param_list(model_names: List[str], model_type:str):
    model_param_list = []
    for name in model_names:
        print(f"loading {name} -----------------")
        model = load_model(name, model_type=model_type)
        model_param_list.append(model.state_dict())
    return model_param_list


### 模型融合

In [ ]:
def merge_param(model_param_list: List[Dict], weights: List[float]):
    new_param = {}
    for k in model_param_list[0].keys():
        for w, param in zip(weights, model_param_list):
            if param[k].dtype == torch.int64 or param[k].dtype == torch.int32:
                new_param[k] = param[k]
            elif k not in new_param:
                new_param[k] = w * param[k]
            else:
                new_param[k] += w * param[k]
    return new_param